<a href="https://colab.research.google.com/github/ayanban011/SwinDocSegmenter/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SwinDocSegmenter**
This notebook try to demonstrate how to use this method efficiently for complex document layout analysis.

## **Step1: Create Environment and Install the necessary packages**

In [1]:
!git clone https://github.com/ayanban011/SwinDocSegmenter.git

Cloning into 'SwinDocSegmenter'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 328 (delta 27), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (328/328), 17.39 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [3]:
%cd SwinDocSegmenter/

/content/SwinDocSegmenter


In [2]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=d1442a9cc4b14dd9bb9fdb7921cdc710bca4abd7eb66a3e12860d80d088fa633
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.6.9 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatibl

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00


In [5]:
%cd maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content/SwinDocSegmenter/maskdino/modeling/pixel_decoder/ops
running build
running build_py
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/functions
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-310/functions
copying functions/__init__.py -> build/lib.linux-x86_64-cpython-310/functions
creating build/lib.linux-x86_64-cpython-310/modules
copying modules/ms_deform_attn.py -> build/lib.linux-x86_64-cpython-310/modules
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-310/modules
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:398: UserWarning: There are no x86_64-linux-gnu-g++ version bounds define

In [6]:
%cd ../../../..

/content/SwinDocSegmenter


Ufff! This is the most critical job we have to done. As it is done. The inference is much easier now.

## **Step2: Inference on test dataset**

In [7]:
#!/usr/bin/env python
# Copyright (c) Facebook, Inc. and its affiliates.

import argparse
import json
import numpy as np
import os
from collections import defaultdict
import cv2
import tqdm

from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import Boxes, BoxMode, Instances
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data.datasets import register_coco_instances

In [12]:
register_coco_instances("prima_val",{},"/content/drive/MyDrive/layout-model-training-master/layout-model-training-master/tools/annotations-val.json","/content/drive/MyDrive/layout-model-training-master/layout-model-training-master/data/prima/val")
input = "/content/coco_instances_results.json"
output = "/content"
conf_threshold = 0.5

In [13]:
with PathManager.open(input, "r") as f:
  predictions = json.load(f)

pred_by_image = defaultdict(list)
for p in predictions:
  pred_by_image[p["image_id"]].append(p)

dicts = list(DatasetCatalog.get("prima_val"))
metadata = MetadataCatalog.get("prima_val")
if hasattr(metadata, "thing_dataset_id_to_contiguous_id"):
  def dataset_id_map(ds_id):
    return metadata.thing_dataset_id_to_contiguous_id[ds_id]

os.makedirs(output, exist_ok=True)

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [15]:
def create_instances(predictions, image_size):
    ret = Instances(image_size)

    score = np.asarray([x["score"] for x in predictions])
    chosen = (score > conf_threshold).nonzero()[0]
    score = score[chosen]
    bbox = np.asarray([predictions[i]["bbox"] for i in chosen]).reshape(-1, 4)
    bbox = BoxMode.convert(bbox, BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)

    labels = np.asarray([dataset_id_map(predictions[i]["category_id"]) for i in chosen])

    ret.scores = score
    ret.pred_boxes = Boxes(bbox)
    ret.pred_classes = labels

    try:
        ret.pred_masks = [predictions[i]["segmentation"] for i in chosen]
    except KeyError:
        pass
    return ret

In [16]:
for dic in tqdm.tqdm(dicts):
  img = cv2.imread(dic["file_name"], cv2.IMREAD_COLOR)[:, :, ::-1]
  basename = os.path.basename(dic["file_name"])
  predictions = create_instances(pred_by_image[dic["image_id"]], img.shape[:2])
  vis = Visualizer(img, metadata)
  vis_pred = vis.draw_instance_predictions(predictions).get_image()
  vis = Visualizer(img, metadata)
  vis_gt = vis.draw_dataset_dict(dic).get_image()
  concat = np.concatenate((vis_pred, vis_gt), axis=1)
  cv2.imwrite(os.path.join(output, basename), concat[:, :, ::-1])

100%|██████████| 91/91 [07:29<00:00,  4.94s/it]


In [19]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64encode
import cv2

def imshow(name, img):
  """Put frame as <img src="data:image/jpg;base64,...."> """

  js = Javascript('''
  async function showImage(name, image, width, height) {
    img = document.getElementById(name);
    if(img == null) {
      img = document.createElement('img');
      img.id = name;
      document.body.appendChild(img);
    }
    img.src = image;
    img.width = width;
    img.height = height;
  }
  ''')

  height, width = img.shape[:2]

  ret, data = cv2.imencode('.jpg', img)   # compress array of pixels to JPG data
  data = b64encode(data)                  # encode base64
  data = data.decode()                    # convert bytes to string
  data = 'data:image/jpg;base64,' + data  # join header ("data:image/jpg;base64,") and base64 data (JPG)

  display(js)
  eval_js(f'showImage("{name}", "{data}", {width}, {height})')  # run JavaScript code to put image (JPG as string base64) in <img>
                                          # `name` and `data` in needs `" "` to send it as text, not as name of variabe.

In [20]:
import os
import cv2
import imutils
import time

folder = r'/content/'

VALID_FORMAT = (".JPG", ".JPEG", ".PNG")

for number, filename in enumerate(os.listdir(folder)):
    if filename.upper().endswith(VALID_FORMAT):
        path = os.path.join(folder, filename)

        image = cv2.imread(path)
        
        frame = imutils.resize(image, width=400)

        number = number % 2        
        imshow(f"img{number}", frame)

        time.sleep(1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>